In [145]:
#LIBRARIES
import pandas as pd
import matplotlib.pyplot as plt

In [146]:
#READ FILE + DROP NULL COLUMNS
original_file = pd.read_excel('RA1.xlsx')
original_file.dropna(how="all", inplace=True)
original_file = original_file.drop(['Unnamed: 1','Unnamed: 28','Unnamed: 29'], axis=1)
# original_file.dropna(axis='columns', how='all')

In [147]:
df = original_file.iloc[3:,:]

In [148]:
df.columns = original_file.iloc[2,:]

In [149]:
#HEADER LIST
colog = list(df.columns.values)
col = [str(i) for i in list(df.columns.values)]
c = col[:]
unclean_column_names = col.copy()
col2 = df.iloc[:1,:].values[0]

for i in col:
    if col.count(i) == 2:
        col[col.index(i)] = i+'-T'
        
i=2
while i < len(col)-3:
    x = col[i-1]
    while 'nan' in col[i]:
        col[i] = x
        i = i+1
    i = i+1
    
#INITIALIZE
col[0] = 'SEAT NUM'
col[i] = 'TOTAL'
col[i+1] = 'GPA'
col[i+2] = 'RESULT'
col2[0] = ''
col2[-1] = ''
col2[-2] = ''
col2[-3] = ''

# print(l)
cols = list(zip(col,col2))
header = ["-".join(i).strip('-') for i in cols]

col.insert(1,'NAME')  #For Storing orignal order of columns

In [150]:
newdf = df.iloc[2:,:]
newdf.columns = original_file.iloc[1,:]
coursenum = [str(i) for i in list(newdf.columns)][1:-3]
coursenum = [i for i in coursenum if 'nan' not in i]
subject_names = [i for i in unclean_column_names if 'nan' not in i]
subjectcodes = dict(zip(coursenum,subject_names))

In [151]:
df.columns = header
df = df.iloc[3:,:]
df.index = pd.RangeIndex(len(df.index))

dfmarks = df.loc[::2]
dfnames = df.loc[1::2]

# dfmarks.loc[:,'NAME'] = dfnames['SEAT NUM'].values
dfmarks.loc[:,'NAME'] = dfnames.loc[:,'SEAT NUM'].values
dfmarks.index = pd.RangeIndex(len(dfmarks.index))

In [152]:
l = [i for i in dfnames.columns if 'TOT' in i][:-1] + list(dfnames.columns[-3:])
dfnames = dfnames.drop(l, axis=1)

In [153]:
def clean(x):
#     if x == 'Ab':
#         return int(0)
    x = list(str(x))
    if x[-1] == 'E' or  x[-1] == 'F':
        return "".join(x[:-1]).strip()
    else:
        return "".join(x)

In [154]:
for i in dfmarks.columns[1:-4]:
    dfmarks.loc[:,i] = dfmarks.loc[:,i].apply(clean)

In [155]:
dfcolumns = list(dfmarks.columns)
totalcols = [dfcolumns[0]] + [i for i in dfcolumns if 'TOT' in i]
totalcols = totalcols + dfcolumns[-3:]
dftotalmarks = dfmarks.loc[:,totalcols]

In [156]:
originalcol = col[:]
originalcol = list( dict.fromkeys(originalcol) ) #Stores Original Order of columns

In [157]:
#REORDER COLUMNS
totalmarkscol = []
for i in originalcol:
    if i in dftotalmarks.columns:
        totalmarkscol.append(i)
    elif str(i+str('-TOT')) in dftotalmarks.columns:
        totalmarkscol.append(str(i+str('-TOT')))

In [158]:
temp = [i for i in coursenum if coursenum[0][:3] in i]

In [159]:
subjectcols = list(dftotalmarks)[1:-4]
subjectcols = subjectcols[:len(temp)]

In [160]:
totalmarkscol = totalmarkscol[:2] + subjectcols + totalmarkscol[-3:]

In [161]:
dftotalmarks = dftotalmarks[totalmarkscol]

In [162]:
dftotalmarks.to_csv('TOTALRESULT.csv')

In [163]:
subjectcols

['APPLIED MATHEMATICS III-TOT',
 'THERMODYNAMICS-TOT',
 'STRENGTH OF MATERIALS-T-TOT',
 'PRODUCTION PROCESS I-TOT',
 'MATERIAL TECHNOLOGY-T-TOT']

In [164]:
def checkpass(x,val):
    if x == 'Ab':
        return 0
    return int(int(x)>val)
def checkrange(x, p,q):
    if x == 'Ab':
        return 0
    return int(p < int(x) and int(x)<q)

In [165]:
passing = pd.DataFrame()

passing['Topper'] = dftotalmarks.loc[:,'GPA'].apply(lambda x: int(float(x)>9.3))

for i in subjectcols:
    passing[i.replace('TOT','PASSED')] = dftotalmarks[i].apply(lambda x :checkpass(x,40))


for i in subjectcols:
    passing[i.replace('TOT','40<X<50')] = dftotalmarks[i].apply(lambda x :checkrange(x,40,50))
    passing[i.replace('TOT','50<X<60')] = dftotalmarks[i].apply(lambda x :checkrange(x,50,60))
    passing[i.replace('TOT','X>60')] = dftotalmarks[i].apply(lambda x :checkrange(x,60,100))
    
passing['GPA < 40'] = dftotalmarks.loc[:,'GPA'].apply(lambda x: int(float(x)<4))
passing['40 < GPA < 50'] = dftotalmarks.loc[:,'GPA'].apply(lambda x: int(float(x)>3.9 and float(x)<5.1))
passing['50 < GPA < 60'] = dftotalmarks.loc[:,'GPA'].apply(lambda x: int(float(x)>5.0 and float(x)<6.1))
passing['60 < GPA < 70'] = dftotalmarks.loc[:,'GPA'].apply(lambda x: int(float(x)>6.0 and float(x)<7.1))
passing['70 < GPA < 80'] = dftotalmarks.loc[:,'GPA'].apply(lambda x: int(float(x)>7.0 and float(x)<8.1))
passing['80 < GPA < 90'] = dftotalmarks.loc[:,'GPA'].apply(lambda x: int(float(x)>8.0 and float(x)<9.1))
passing['90 < GPA < 100'] = dftotalmarks.loc[:,'GPA'].apply(lambda x: int(float(x)>9.0 and float(x)<10.1))

passing['Perfect 100'] = dftotalmarks.loc[:,'GPA'].apply(lambda x: int(float(x) == 10))

In [166]:
type(passing)

pandas.core.frame.DataFrame

In [167]:
passing

,Topper,APPLIED MATHEMATICS III-PASSED,THERMODYNAMICS-PASSED,STRENGTH OF MATERIALS-T-PASSED,PRODUCTION PROCESS I-PASSED,MATERIAL TECHNOLOGY-T-PASSED,APPLIED MATHEMATICS III-40<X<50,APPLIED MATHEMATICS III-50<X<60,APPLIED MATHEMATICS III-X>60,THERMODYNAMICS-40<X<50,...,MATERIAL TECHNOLOGY-T-50<X<60,MATERIAL TECHNOLOGY-T-X>60,GPA < 40,40 < GPA < 50,50 < GPA < 60,60 < GPA < 70,70 < GPA < 80,80 < GPA < 90,90 < GPA < 100,Perfect 100
0,0,0,0,0,1,1,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,0,1,0,1,1,1,1,0,0,0,...,1,0,1,0,0,0,0,0,0,0
2,0,1,1,1,1,1,0,0,1,0,...,0,1,0,0,0,0,1,0,0,0
3,0,0,0,1,1,1,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4,0,0,0,0,1,1,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
5,0,1,1,1,1,1,1,0,0,0,...,0,1,0,0,0,0,1,0,0,0
6,1,1,1,1,1,1,0,0,1,0,...,0,1,0,0,0,0,0,0,1,0
7,0,1,1,1,1,1,0,0,1,0,...,0,1,0,0,0,0,1,1,0,0
8,0,1,1,1,1,1,0,0,1,1,...,1,0,0,0,0,0,1,0,0,0
9,0,1,1,1,1,1,0,0,1,0,...,0,1,0,0,0,0,1,0,0,0


In [168]:
float(10.0) == 10

True

In [169]:
passing.to_excel('EXCEL-ANALYSIS.xlsx')